In [ ]:
#import os
#print("CWD:", os.getcwd())


CWD: c:\Users\tahir\CS484\project


In [4]:
!pip install polars 
!pip install lightgbm
!pip install pandas
!pip install scikit-learn
!pip install pyarrow


In [1]:
#imports
import os
import glob
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

TRAIN_PARQ_DIR = "./csv_files/train/parquet/"
TEST_PARQ_DIR = "./csv_files/test/parquet/"


In [2]:
#loading base tables
train_base = pl.read_parquet(TRAIN_PARQ_DIR + "train_base.parquet")
test_base  = pl.read_parquet(TEST_PARQ_DIR + "test_base.parquet")

print("train_base:", train_base.shape)
print("test_base:", test_base.shape)


train_base: (1526659, 5)
test_base: (10, 4)


In [9]:
import polars as pl

def aggregate_by_case(df: pl.DataFrame, prefix: str) -> pl.DataFrame:
    """
    Aggregate a long table to one row per case_id.
    Numeric columns -> mean, std, min, max, sum, count.
    Uses only Polars (no pandas / pyarrow).
    """

    # treat all integer, float, and boolean columns as numeric
    numeric_types = (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64,
        pl.Boolean,
    )

    numeric_cols = []
    for c, dt in zip(df.columns, df.dtypes):
        if c == "case_id":
            continue
        if dt in numeric_types:
            numeric_cols.append(c)

    print(f"[{prefix}] numeric columns: {len(numeric_cols)}")

    aggs = []
    for c in numeric_cols:
        aggs.extend([
            pl.col(c).mean().alias(f"{prefix}__{c}__mean"),
            pl.col(c).std().alias(f"{prefix}__{c}__std"),
            pl.col(c).min().alias(f"{prefix}__{c}__min"),
            pl.col(c).max().alias(f"{prefix}__{c}__max"),
            pl.col(c).sum().alias(f"{prefix}__{c}__sum"),
        ])

    # how many rows contributed for this case_id
    aggs.append(pl.len().alias(f"{prefix}__row_count"))

    out = df.group_by("case_id").agg(aggs)
    print(f"[{prefix}] aggregated shape: {out.shape}")
    return out


In [4]:
def load_parquet_group(pattern: str, prefix: str) -> pl.DataFrame | None:
    """
    Glob parquet files and concat them.
    We only use this for applprev1 & applprev2, which are already known to work.
    """
    files = glob.glob(pattern)
    print(f"[{prefix}] pattern: {pattern} -> {len(files)} files")

    if not files:
        print(f"[{prefix}] WARNING: no files found, skipping.\n")
        return None

    dfs = [pl.read_parquet(f) for f in files]
    out = pl.concat(dfs, how="vertical_relaxed")
    print(f"[{prefix}] raw shape: {out.shape}")
    return out


In [49]:
TRAIN_PARQ_DIR = "./csv_files/train/parquet/"
TEST_PARQ_DIR = "./csv_files/test/parquet/"

groups = [
    ("applprev1",
     TRAIN_PARQ_DIR + "train_applprev_1_*.parquet",
     TEST_PARQ_DIR  + "test_applprev_1_*.parquet"),

    ("applprev2",
     TRAIN_PARQ_DIR + "train_applprev_2.parquet",
     TEST_PARQ_DIR  + "test_applprev_2.parquet"),

    ("creditA",
     TRAIN_PARQ_DIR + "train_credit_bureau_a_*.parquet",
     TEST_PARQ_DIR  + "test_credit_bureau_a_*.parquet"),

    ("creditB1",
     TRAIN_PARQ_DIR + "train_credit_bureau_b_1.parquet",
     TEST_PARQ_DIR  + "test_credit_bureau_b_1.parquet"),

    ("creditB2",
     TRAIN_PARQ_DIR + "train_credit_bureau_b_2.parquet",
     TEST_PARQ_DIR  + "test_credit_bureau_b_2.parquet"),

    ("debit1",
     TRAIN_PARQ_DIR + "train_debitcard_1.parquet",
     TEST_PARQ_DIR  + "test_debitcard_1.parquet"),

    ("deposit1",
     TRAIN_PARQ_DIR + "train_deposit_1.parquet",
     TEST_PARQ_DIR  + "test_deposit_1.parquet"),

    ("other1",
     TRAIN_PARQ_DIR + "train_other_1.parquet",
     TEST_PARQ_DIR  + "test_other_1.parquet"),

    ("taxA1",
     TRAIN_PARQ_DIR + "train_tax_registry_a_1.parquet",
     TEST_PARQ_DIR  + "test_tax_registry_a_1.parquet"),

    ("taxB1",
     TRAIN_PARQ_DIR + "train_tax_registry_b_1.parquet",
     TEST_PARQ_DIR  + "test_tax_registry_b_1.parquet"),

    ("taxC1",
     TRAIN_PARQ_DIR + "train_tax_registry_c_1.parquet",
     TEST_PARQ_DIR  + "test_tax_registry_c_1.parquet"),
]


In [50]:
train_df = train_base
test_df  = test_base

for prefix, train_pattern, test_pattern in groups:
    print(f"\n=== Processing group: {prefix} ===")
    
    train_long = load_parquet_group(train_pattern, prefix + "_train")
    test_long  = load_parquet_group(test_pattern,  prefix + "_test")

    if train_long is None or test_long is None:
        print(f"[{prefix}] skipped (files missing)")
        continue

    train_agg = aggregate_by_case(train_long, prefix)
    test_agg  = aggregate_by_case(test_long,  prefix)

    # left join on case_id
    train_df = train_df.join(train_agg, on="case_id", how="left")
    test_df  = test_df.join(test_agg,  on="case_id", how="left")

    print(f"[{prefix}] train_df shape now: {train_df.shape}")
    print(f"[{prefix}] test_df  shape now: {test_df.shape}")




=== Processing group: applprev1 ===
[applprev1_train] pattern: ./csv_files/train/parquet/train_applprev_1_*.parquet -> 2 files
[applprev1_train] raw shape: (6525979, 41)
[applprev1_test] pattern: ./csv_files/test/parquet/test_applprev_1_*.parquet -> 3 files
[applprev1_test] raw shape: (30, 41)
[applprev1] numeric columns: 21
[applprev1] aggregated shape: (1221522, 107)
[applprev1] numeric columns: 21
[applprev1] aggregated shape: (4, 107)
[applprev1] train_df shape now: (1526659, 111)
[applprev1] test_df  shape now: (10, 110)

=== Processing group: applprev2 ===
[applprev2_train] pattern: ./csv_files/train/parquet/train_applprev_2.parquet -> 1 files
[applprev2_train] raw shape: (14075487, 6)
[applprev2_test] pattern: ./csv_files/test/parquet/test_applprev_2.parquet -> 1 files
[applprev2_test] raw shape: (10, 6)
[applprev2] numeric columns: 2
[applprev2] aggregated shape: (1221522, 12)
[applprev2] numeric columns: 2
[applprev2] aggregated shape: (1, 12)
[applprev2] train_df shape now: 

ComputeError: schema lengths differ

In [38]:
print("Polars shapes:", train_df.shape, test_df.shape)

train_pd = pd.DataFrame(train_df.to_dicts())
test_pd  = pd.DataFrame(test_df.to_dicts())

print("Pandas shapes:", train_pd.shape, test_pd.shape)
train_pd.head()


Polars shapes: (1526659, 122) (10, 121)
Pandas shapes: (1526659, 122) (10, 121)


,case_id,date_decision,MONTH,WEEK_NUM,target,applprev1__actualdpd_943P__mean,applprev1__actualdpd_943P__std,applprev1__actualdpd_943P__min,applprev1__actualdpd_943P__max,applprev1__actualdpd_943P__sum,...,applprev2__num_group1__std,applprev2__num_group1__min,applprev2__num_group1__max,applprev2__num_group1__sum,applprev2__num_group2__mean,applprev2__num_group2__std,applprev2__num_group2__min,applprev2__num_group2__max,applprev2__num_group2__sum,applprev2__row_count
0,0,2019-01-03,201901,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-01-03,201901,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019-01-04,201901,0,0,0.0,0.0,0.0,0.0,0.0,...,0.57735,0.0,1.0,2.0,0.5,0.577350,0.0,1.0,2.0,4.0
3,3,2019-01-03,201901,0,0,0.0,NaN,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,1.0,1.000000,0.0,2.0,3.0,3.0
4,4,2019-01-04,201901,0,1,0.0,NaN,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.5,0.707107,0.0,1.0,1.0,2.0


In [39]:
sample = pd.read_csv("./csv_files/sample_submission.csv")
print(sample.head())

ID_COL = sample.columns[0]    # e.g., 'case_id'
TARGET_COL = "target"         # per competition spec

print("ID_COL:", ID_COL, "TARGET_COL:", TARGET_COL)


   case_id  score
0    57543    0.5
1    57549    0.5
2    57551    0.5
3    57552    0.5
4    57569    0.5
ID_COL: case_id TARGET_COL: target


In [40]:
import numpy as np
drop_cols = [ID_COL, TARGET_COL]

# optional: drop date-like columns if they exist
for col in ["date_decision", "WEEK_NUM"]:
    if col in train_pd.columns:
        drop_cols.append(col)

numeric_cols = train_pd.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in drop_cols]

print("Num features:", len(feature_cols))

X = train_pd[feature_cols]
y = train_pd[TARGET_COL]
X_test = test_pd[feature_cols]

print("X:", X.shape, "X_test:", X_test.shape)


Num features: 114
X: (1526659, 114) X_test: (10, 114)


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    precision_recall_curve,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    average_precision_score,
)


In [42]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Val:", X_val.shape)


Train: (1221327, 114) Val: (305332, 114)


In [43]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data   = lgb.Dataset(X_val, label=y_val)

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 63,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "verbose": -1,
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    valid_names=["train", "val"],
    num_boost_round=2000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=50),
    ],
)

val_pred = model.predict(X_val, num_iteration=model.best_iteration)
val_auc  = roc_auc_score(y_val, val_pred)
print("Validation AUC:", val_auc)


Training until validation scores don't improve for 100 rounds
[50]	train's auc: 0.743465	val's auc: 0.722636
[100]	train's auc: 0.760512	val's auc: 0.727661
[150]	train's auc: 0.774097	val's auc: 0.729171
[200]	train's auc: 0.784999	val's auc: 0.729505
[250]	train's auc: 0.794169	val's auc: 0.729858
[300]	train's auc: 0.803342	val's auc: 0.729933
[350]	train's auc: 0.810145	val's auc: 0.729914
Early stopping, best iteration is:
[275]	train's auc: 0.798736	val's auc: 0.730099
Validation AUC: 0.7300994589091913


In [44]:
# Precision-recall curve to choose threshold
precision, recall, thresholds = precision_recall_curve(y_val, val_pred)

# Avoid division by zero
f1s = 2 * (precision * recall) / (precision + recall + 1e-9)
best_idx = np.argmax(f1s)
best_threshold = thresholds[best_idx]

print("Best threshold:", best_threshold)

# Metrics at default 0.5
val_bin_05 = (val_pred > 0.5).astype(int)
acc_05 = accuracy_score(y_val, val_bin_05)
rec_05 = recall_score(y_val, val_bin_05)
prec_05 = precision_score(y_val, val_bin_05)
f1_05 = f1_score(y_val, val_bin_05)

print("\n=== Metrics at threshold 0.5 ===")
print("Accuracy:", acc_05)
print("Recall:",   rec_05)
print("Precision:",prec_05)
print("F1:",       f1_05)

# Metrics at best_threshold
val_bin_best = (val_pred > best_threshold).astype(int)
acc_best = accuracy_score(y_val, val_bin_best)
rec_best = recall_score(y_val, val_bin_best)
prec_best = precision_score(y_val, val_bin_best)
f1_best = f1_score(y_val, val_bin_best)

print("\n=== Metrics at best threshold ===")
print("Accuracy:", acc_best)
print("Recall:",   rec_best)
print("Precision:",prec_best)
print("F1:",       f1_best)

# PR-AUC
pr_auc = average_precision_score(y_val, val_pred)
print("\nPR-AUC:", pr_auc)


Best threshold: 0.08348859070624812

=== Metrics at threshold 0.5 ===
Accuracy: 0.9685653649142573
Recall: 0.0007292426294405667
Precision: 0.5384615384615384
F1: 0.0014565126924677486

=== Metrics at best threshold ===
Accuracy: 0.9212365556181468
Recall: 0.25523492030419836
Precision: 0.12661498708010335
F1: 0.16926318698400636

PR-AUC: 0.09903058380934145


In [45]:
test_pred = model.predict(X_test, num_iteration=model.best_iteration)

submission = pd.DataFrame({
    ID_COL: test_pd[ID_COL],
    "score": test_pred,
})

submission.to_csv("submission_full_parquet_model.csv", index=False)
submission.head()


,case_id,score
0,57543,0.036672
1,57549,0.027360
2,57551,0.014639
3,57552,0.014639
4,57569,0.014639


In [47]:
precision, recall, thresholds = precision_recall_curve(y_val, val_pred)
f1s = 2 * (precision * recall) / (precision + recall + 1e-9)
best_idx = np.argmax(f1s)
best_threshold = thresholds[best_idx]
print("Best threshold:", best_threshold)

val_bin_best = (val_pred > best_threshold).astype(int)
print("Accuracy:",      accuracy_score(y_val, val_bin_best))
print("Recall:",        recall_score(y_val, val_bin_best))
print("Precision:",     precision_score(y_val, val_bin_best))
print("F1:",            f1_score(y_val, val_bin_best))
print("PR-AUC:",        average_precision_score(y_val, val_pred))


Best threshold: 0.08348859070624812
Accuracy: 0.9212365556181468
Recall: 0.25523492030419836
Precision: 0.12661498708010335
F1: 0.16926318698400636
PR-AUC: 0.09903058380934145
